## Testing data splits

In [1]:
import os
import json
import numpy as np

data_path = "../../data/"

%load_ext autoreload
%autoreload 2

from llm.llm_utils import (
    split_and_save_data,
    load_data_as_lists
)

In [13]:
# save data splits
# raw_data_path = data_path + "raw/generator=base~dataset=p1.00.jsonl"
# split_and_save_data(raw_data_path)

In [2]:
# load data
train_data = load_data_as_lists(data_path + "splits/train.jsonl")
val_data = load_data_as_lists(data_path + "splits/val.jsonl")
test_data = load_data_as_lists(data_path + "splits/test.jsonl")

print(train_data[0][:2])
print(train_data[1][:2])  # 0 corresponds to human, 1 corresponds to machine

["Patrons crowd the platform at the Washington Metropolitan Area Transit Authority's (WMATA) Metro Center stop in Washington, D.C. on Dec. 20, 2004. Thousands use the public transit system daily to get them in and around the D.C. area. (Karen Bleier/AFP/Getty Images)\nUS Senators Threaten Metro Funding Over Chinese Manufacturer\nWASHINGTON —Federal lawmakers say they’ll approve badly needed funding for Washington’s transit system, but only if it avoids buying new rail cars from China.\nThe Washington Post reported on April 13 that U.S. Senators from Virginia and Maryland proposed the idea in new legislation. It reflects growing concerns that China’s state-owned rail company could hurt American manufacturers and make the system vulnerable to cyber espionage.\nDave Smolensky, spokesman for the China Railway Rolling Stock Corp, dismissed the espionage concerns. The company also said the United States should be promoting competition.\nThe company has won four major U.S. rail car contracts.

In [3]:
print(f"Num of train samples: {len(train_data[0])}")
print(f"Num of test samples: {len(test_data[0])}")
print(f"Num of val samples: {len(val_data[0])}")
# why are there 12k test samples wtf?

Num of train samples: 10000
Num of test samples: 12000
Num of val samples: 3000


## Testing Dataset functionality

In [23]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
data_files = {
    "train": data_path + "splits/train.jsonl",
    "val" : data_path + "splits/val.jsonl",
    "test": data_path + "splits/test.jsonl",
}
grover_dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
grover_dataset["train"][0]

{'article': "Patrons crowd the platform at the Washington Metropolitan Area Transit Authority's (WMATA) Metro Center stop in Washington, D.C. on Dec. 20, 2004. Thousands use the public transit system daily to get them in and around the D.C. area. (Karen Bleier/AFP/Getty Images)\nUS Senators Threaten Metro Funding Over Chinese Manufacturer\nWASHINGTON —Federal lawmakers say they’ll approve badly needed funding for Washington’s transit system, but only if it avoids buying new rail cars from China.\nThe Washington Post reported on April 13 that U.S. Senators from Virginia and Maryland proposed the idea in new legislation. It reflects growing concerns that China’s state-owned rail company could hurt American manufacturers and make the system vulnerable to cyber espionage.\nDave Smolensky, spokesman for the China Railway Rolling Stock Corp, dismissed the espionage concerns. The company also said the United States should be promoting competition.\nThe company has won four major U.S. rail car

In [28]:
# article length analysis
lens = {"train": [], "val": [], "test": []}

for split in grover_dataset.keys():
    for i in range(len(grover_dataset[split])):
        text = grover_dataset[split][i]['article']
        lens[split].append(len(text.split()))

print(sorted(lens["train"], reverse=True)[:10])
# plt.hist(lens['train'], bins=50)
print(sorted(lens["val"], reverse=True)[:10])
# plt.hist(lens['val'], bins=50)
print(sorted(lens["test"], reverse=True)[:10])
# plt.hist(lens['test'], bins=50)

#### The longest article seems to be 13k words!


[13705, 11993, 7946, 7401, 7060, 5119, 4879, 4442, 4162, 4077]
[7413, 5297, 4729, 3930, 3847, 3476, 3227, 3034, 3007, 2912]
[11333, 11231, 10991, 9628, 8623, 6146, 6126, 5461, 5367, 5255]


In [29]:
# data imbalance checking
label_counts = {}
for split in grover_dataset.keys():
    df = pd.DataFrame(grover_dataset[split])
    label_counts[split] = df['label'].value_counts()
print(label_counts)

# weird, train is balanced, but val and test are imbalanced.

{'train': label
human      5000
machine    5000
Name: count, dtype: int64, 'val': label
human      2000
machine    1000
Name: count, dtype: int64, 'test': label
human      8000
machine    4000
Name: count, dtype: int64}
